In [1]:
import pandas as pd
import numpy as np
import collections
import copy
import random
import re
import matplotlib.pyplot as plt
from nlp.utils import (
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)
from item.utils import get_tokens_set
import seaborn as sns

sns.set()

# Load price statistics

In [2]:
prices = pd.read_csv('../dados/output/cluster_prices_statistics.csv.zip', sep=';')
items = pd.read_csv("../dados/output/items_clusters_train.csv.zip", sep=';')

In [3]:
len(prices[['dsc_unidade_medida', 'cluster']].drop_duplicates())

196630

In [4]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196630 entries, 0 to 196629
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   cluster             196630 non-null  object 
 1   dsc_unidade_medida  196544 non-null  object 
 2   mean                196630 non-null  float64
 3   count               196630 non-null  int64  
 4   max                 196630 non-null  float64
 5   min                 196630 non-null  float64
 6   median              196630 non-null  float64
 7   std                 150596 non-null  float64
 8   var                 150596 non-null  float64
 9   quantile_1          196630 non-null  float64
 10  quantile_3          196630 non-null  float64
dtypes: float64(8), int64(1), object(2)
memory usage: 16.5+ MB


In [5]:
prices['first_token'] = prices['cluster'].str.split('_').str[0]

In [6]:
prices['std'] = prices['std'].fillna(0.0)
prices['var'] = prices['var'].fillna(0.0)
prices['mean'] = prices['mean'].fillna(1.0)

In [7]:
prices = prices.rename({'count': 'n_items'}, axis=1)

In [8]:
prices.head()

,cluster,dsc_unidade_medida,mean,n_items,max,min,median,std,var,quantile_1,quantile_3,first_token
0,a100_0,tb,56.000000,2,56.00,56.00,56.00,0.000000,0.000000,56.000,56.000,a100
1,a100_0,tubo,46.150000,2,60.10,32.20,46.15,19.728279,389.205000,39.175,53.125,a100
2,a100_0,tubos,50.550000,1,50.55,50.55,50.55,0.000000,0.000000,50.550,50.550,a100
3,a100_0,unidade,46.446615,65,75.75,32.57,44.67,13.922649,193.840151,32.840,55.000,a100
4,a100_0,unidades,40.480000,1,40.48,40.48,40.48,0.000000,0.000000,40.480,40.480,a100


In [9]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4448777 entries, 0 to 4448776
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   item_id             int64  
 1   seq_dim_licitacao   int64  
 2   outlier             int64  
 3   cluster             object 
 4   dsc_unidade_medida  object 
 5   ano                 int64  
 6   description         object 
 7   original            object 
 8   areas               object 
 9   price               float64
dtypes: float64(1), int64(4), object(5)
memory usage: 339.4+ MB


In [10]:
# remove outliers
items = items[items.outlier != 1]

In [11]:
items['first_token'] = items['cluster'].str.split('_').str[0]

In [12]:
items.head()

,item_id,seq_dim_licitacao,outlier,cluster,dsc_unidade_medida,ano,description,original,areas,price,first_token
150,4010,716236,0,achocolatar_24,pacote,2017,achocolatar po 400 gr base acucar cacau maltod...,"Achocolatado em pó de 400gr - ACHOCOLATADO, em...",Vazio,4.33,achocolatar
151,4951,716287,0,achocolatar_24,pacote,2018,achocolatar po 400 gr base acucar cacau maltod...,"Achocolatado em pó de 400gr - ACHOCOLATADO, em...",Vazio,3.88,achocolatar
152,31955,722115,0,achocolatar_24,pacote,2017,achocolatar po ii embalagem 400 g composicao a...,Achocolatado em po II - embalagem 400 g. Compo...,Assistência Social | Educação,2.99,achocolatar
153,158513,749927,0,achocolatar_24,unidade,2015,achocolatar po diet 39 menos caloria 35 cacau ...,ACHOCOLATADO EM PO DIET - Achocolatado em pó d...,Vazio,10.35,achocolatar
154,161886,750062,0,achocolatar_24,unidade,2018,achocolatar po diet 39 menos caloria 35 cacau ...,ACHOCOLATADO EM PO DIET - Achocolatado em pó d...,Vazio,10.90,achocolatar


In [13]:
items_df = pd.merge(items, prices.drop('first_token', axis=1), how='left', left_on=['cluster','dsc_unidade_medida'], right_on = ['cluster','dsc_unidade_medida'])

## Print clusters info

In [14]:
def print_clusters_info(cluster_name, min_size=20, total_unit_metrics=5, total_clusters=5, total_items=5, total_words=20):
    
    clusters = prices[prices.first_token == cluster_name][['cluster', 'dsc_unidade_medida', 'n_items']]
    clusters_ranking = clusters.sort_values(by='n_items', ascending=False)
    
    print("#itens:", sum(list(clusters["n_items"])))
    print("#grupos:", len(clusters_ranking))
    print("Grupos com +20 itens:", len(clusters_ranking[clusters_ranking.n_items >= min_size]))
    print("#itens/#grupos: {:.2f}".format(sum(list(clusters["n_items"]))/len(clusters_ranking)))

    top_unit_metrics = clusters.groupby("dsc_unidade_medida").sum().sort_values(by='n_items', ascending=False).head(total_unit_metrics).reset_index()
    
    print()
    print("Unidades de medida mais frequentes:")
    for unit in top_unit_metrics.iterrows():
        print("{} ({:.1f}%)".format(unit[1]['dsc_unidade_medida'], (unit[1]['n_items']/sum(list(clusters["n_items"])))*100))
    print()
        
    clusters_ranking = clusters_ranking[clusters_ranking.n_items >= min_size].head(total_clusters)
    
    for cluster_id in clusters_ranking.index.values:
        cluster_info = clusters.loc[cluster_id]
        cluster_items = items[(items.cluster == cluster_info['cluster']) & (items.dsc_unidade_medida == cluster_info['dsc_unidade_medida'])].copy()
        cluster_items["description_tokens"] = cluster_items.description.apply(lambda d: d.split())
        top_descriptions = cluster_items.groupby("description").size().sort_values(ascending=False).head(total_items)
        present_tokens = collections.Counter()
        for tokens in cluster_items["description_tokens"]:
            present_tokens.update(tokens)

        present_tokens = list(present_tokens.items())
        present_tokens.sort(key= lambda x : x[1], reverse=True)
        present_tokens = [token[0] for token in present_tokens][:total_words]

        print("Cluster {} ({}): ".format(cluster_info['cluster'], cluster_info['dsc_unidade_medida']))
        print("Tamanho: {}".format(len(cluster_items)))
        print("Descrições únicas: {}".format(len(set(cluster_items["description"]))))
        print("{} Termos mais frequentes: {}".format(total_words, present_tokens))
        print("Descrições mais frequentes:")
        for desc in top_descriptions.iteritems():
            print("{} ({:.1f}%)".format(desc[0], (desc[1]/len(cluster_items))*100))
        print("\n")

# Dipirona

In [29]:
print_clusters_info("dipirona")

#itens: 3379
#grupos: 321
Grupos com +20 itens: 44
#itens/#grupos: 10.53

Unidades de medida mais frequentes:
unidade (26.0%)
frasco (23.4%)
comprimido (19.1%)
ampola (12.2%)
amp (3.9%)

Cluster dipirona_9 (comprimido): 
Tamanho: 327
Descrições únicas: 22
20 Termos mais frequentes: ['dipirona', '500', 'mg', 'comprimido', 'comp', 'compr', 'cpr', 'cp', '129', '8974', '4731', '345', '0017', '959', '9', 'un', '966', 'epp', 'medic', 'fracionar']
Descrições mais frequentes:
dipirona 500 mg (60.6%)
dipirona 500 mg comprimido (15.0%)
dipirona 500 mg comp (7.3%)
dipirona 500 mg compr (3.1%)
dipirona 500 mg cpr (2.1%)


Cluster dipirona_2 (comprimido): 
Tamanho: 248
Descrições únicas: 18
20 Termos mais frequentes: ['dipirona', 'sodico', 'mg', '500', 'comprimido', 'comp', 'compr', 'cpr', 'cp', '9', '5', '250', 'com', 'ct', 'bl', 'al', 'plas', 'inc', 'x', '240']
Descrições mais frequentes:
dipirona sodico 500 mg (60.9%)
dipirona sodico 500 mg comprimido (23.0%)
dipirona sodico 500 mg comp (3.6%)
d

In [ ]:
dipirona = items_df[items_df.first_token == 'dipirona']

In [ ]:
set(dipirona[(dipirona.cluster == 'dipirona_9') & (dipirona.dsc_unidade_medida == 'comprimido')]['description'])

In [ ]:
dipirona[(dipirona.cluster == 'dipirona_0') & (dipirona.dsc_unidade_medida == 'ampola')]

# Pneu

In [15]:
print_clusters_info("pneu")

#itens: 43417
#grupos: 660
Grupos com +20 itens: 338
#itens/#grupos: 65.78

Unidades de medida mais frequentes:
unidade (95.0%)
peca (2.9%)
unidades (1.4%)
serv (0.2%)
pca (0.1%)

Cluster pneu_46 (unidade): 
Tamanho: 11916
Descrições únicas: 1997
20 Termos mais frequentes: ['pneu', 'r', '70', '75', '16', '15', '5', '205', 'x', 'r15', 'r14', '185', '225', '17', '215', '80', '65', 'r16', '195', '20']
Descrições mais frequentes:
pneu 175 70 r14 (2.9%)
pneu 215 75 r 17 5 (2.6%)
pneu 215 75 17 5 (2.4%)
pneu 205 70 r15 (1.9%)
pneu 205 70 r 15 (1.9%)


Cluster pneu_44 (unidade): 
Tamanho: 1868
Descrições únicas: 286
20 Termos mais frequentes: ['pneu', '14', 'r', '70', '175', '185', 'x', '65', 'c', '9', 'radial', 'lona', 't', '24', '28', '80', '8', '88', '20', '60']
Descrições mais frequentes:
pneu 175 70 r 14 (14.8%)
pneu 175 70 14 (7.9%)
pneu 185 r 14 (6.4%)
pneu 185 70 r 14 (6.3%)
pneu 175 65 r 14 (4.2%)


Cluster pneu_188 (unidade): 
Tamanho: 1727
Descrições únicas: 409
20 Termos mais freq

In [ ]:
pneu = items_df[items_df.first_token == 'pneu']

In [ ]:
set(pneu[(pneu.cluster == 'pneu_46') & (pneu.dsc_unidade_medida == 'unidade')]['description'])

In [ ]:
pneu[(pneu.cluster == 'pneu_1') & (pneu.dsc_unidade_medida == 'unidade')]

# Gasolina

In [31]:
print_clusters_info("gasolina")

#itens: 2875
#grupos: 36
Grupos com +20 itens: 12
#itens/#grupos: 79.86

Unidades de medida mais frequentes:
litro (96.1%)
unidade (2.6%)
litro(s) (0.2%)
gab. pref. (0.1%)
litro_s (0.1%)

Cluster gasolina_7 (litro): 
Tamanho: 1738
Descrições únicas: 70
20 Termos mais frequentes: ['gasolina', 'comum', 'automotivo', 'aditivado', '6465', 'tipo', 'c', '4807', '4', '8', '5958', 'social', 'r1', '6', 'assist', 'comumgasolina', '15', 'epp', 'r2', 'aut']
Descrições mais frequentes:
gasolina comum (83.8%)
gasolina automotivo comum (3.0%)
gasolina comum aditivado 6465 (1.3%)
gasolina automotivo tipo comum (1.0%)
gasolina comum 4807 (1.0%)


Cluster gasolina_2 (litro): 
Tamanho: 471
Descrições únicas: 23
20 Termos mais frequentes: ['gasolina', 'aditivado', 'automotivo', 'comun', 'c', 'tipo', 'activada', '1766', 'comuma', '134', 'common', '812', '033', '17', '8', '88', 'adm', '4933', '3112', '31']
Descrições mais frequentes:
gasolina (62.0%)
gasolina aditivado (15.9%)
gasolina comun (7.0%)
gasolina

# Máscara

In [32]:
print_clusters_info("mascara")

#itens: 8007
#grupos: 349
Grupos com +20 itens: 87
#itens/#grupos: 22.94

Unidades de medida mais frequentes:
unidade (58.7%)
caixa (29.2%)
pacote (4.0%)
peca (1.8%)
kit (1.6%)

Cluster mascara_26 (caixa): 
Tamanho: 1304
Descrições únicas: 351
20 Termos mais frequentes: ['mascara', 'descartavel', 'elastico', '50', 'c', 'com', 'cirurgico', 'unidade', 'caixa', 'cx', '100', 'desc', 'unid', 'branco', 'descartar', 'und', 'un', 'tnt', 'cor', 'descarar']
Descrições mais frequentes:
mascara descartavel (11.4%)
mascara cirurgico descartavel (7.4%)
mascara cirurgico (2.5%)
mascara descartavel com elastico (2.5%)
mascara descartavel c elastico (2.4%)


Cluster mascara_26 (unidade): 
Tamanho: 925
Descrições únicas: 312
20 Termos mais frequentes: ['mascara', 'descartavel', 'com', 'cirurgico', 'elastico', '50', 'a95', 'n', '95', 'c', 'nebulizacao', 'unidade', 'pff', 'p', 'azul', 'respirador', 'respiratorio', 'caixa', 'facial', 'venturi']
Descrições mais frequentes:
mascara descartavel (9.2%)
mascara

# Locação

In [34]:
print_clusters_info("locacao")

#itens: 10733
#grupos: 1596
Grupos com +20 itens: 122
#itens/#grupos: 6.72

Unidades de medida mais frequentes:
unidade (38.1%)
diaria (11.6%)
hora (7.5%)
sv (3.4%)
mes (3.0%)

Cluster locacao_38 (unidade): 
Tamanho: 207
Descrições únicas: 18
20 Termos mais frequentes: ['banheiro', 'quimico', 'locacao', 'unidade', 'individual', 'equipamento', '978', '3939', 'polietileno', 'portatil', 'para', 'p', 'n', 'adaptado', 'carnaval', 'unid', 'plastico', 'def', 'deficiente', '4183']
Descrições mais frequentes:
locacao banheiro quimico (76.8%)
locacao banheiro quimico unidade (4.8%)
locacao banheiro quimico individual (2.4%)
locacao banheiro quimico 3939 (1.9%)
locacao banheiro quimico 978 (1.9%)


Cluster locacao_22 (unidade): 
Tamanho: 194
Descrições únicas: 50
20 Termos mais frequentes: ['locacao', 'tenda', 'x', '6', 'm', '4', '5', 'barraca', '8', 'piramidal', 'lona', 'unidade', 'tipo', 'mts', 'tamanho', 'i', 'tensionado', 'mx', 'matalon', 'medir']
Descrições mais frequentes:
locacao tenda (25

# Veículo

In [35]:
print_clusters_info("veiculo")

#itens: 3722
#grupos: 180
Grupos com +20 itens: 44
#itens/#grupos: 20.68

Unidades de medida mais frequentes:
unidade (79.2%)
sv (3.7%)
km (3.5%)
hora (2.1%)
kilometro (2.0%)

Cluster veiculo_35 (unidade): 
Tamanho: 1104
Descrições únicas: 911
20 Termos mais frequentes: ['veiculo', 'minimo', 'porta', 'ano', '1', '4', 'km', 'eletrico', '0', 'com', 'cor', '5', 'direcao', 'hidraulico', 'branco', 'modelo', 'motor', 'capacidade', 'ar', 'zero']
Descrições mais frequentes:
veiculo novo zero quilometro capacidade para cinco passageiro quatro porta cor branco motor 1 0 flex fabricacao 2014 2015 (1.0%)
veiculo popular zero quilometro 4 quatro porta direcao hidraulico flex cor branco para programa bolsa familia (0.7%)
veiculo automotor transporte passageiro com seguinte caracteristico ano 2013 modelo zero km 5 cinco lugar incluido motorista cor solido branco motor 1 0 refrigeracao agua 4 cilindro linha injecao eletronico potencia minimo 78 cv 8 oito valvula transmissao manual quatro marcha frente